In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
url = "https://www.imdb.com/list/ls566941243/"
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.text, "html")

In [5]:
with open("imdb.txt", "w", encoding="UTF-8") as file:
    file.write(str(soup))
print("Soup has been extracted to imdb.txt.")

Soup has been extracted to imdb.txt.


In [23]:
movies_data = []
for movie in soup.find_all("div", class_="lister-item-content"):
    title = movie.find("a").text # Movie Titles for all in the page
    genre = movie.find("span", class_="genre").text.strip() # Genre of the movies
    stars = movie.find("div", class_="ipl-rating-star").find("span", class_='ipl-rating-star__rating').text # Star rating of the movie
    runtime = movie.find("span", class_="runtime").text # Duration of the Movie
    meta_div = movie.find("div", class_="inline-block ratings-metascore") # Metascore Division

    if meta_div:
        '''
        This is to extract the metascores of each movie if the meta division is found for the movie, if not the 
        metascore is set to "N/A"
        '''
        found_span = meta_div.find("span", class_="metascore mixed")
        if found_span:
            metascore = found_span.text
        else:
            metascore = "N/A"
    else:
        metascore = "N/A"

    votes = movie.find("span", text="Votes:").find_next("span").text # Votes given for the movie

    gross_element = movie.find('span', text='Gross:')
    gross_value = gross_element.find_next('span').text if gross_element else "N/A" # The gross value of a movie in USD, N/A if not found.
    # gross_value = movie.find('span', text='Gross:').find_next('span').text
    movies_data.append([title, genre, stars, runtime, metascore, votes, gross_value])

In [24]:
df = pd.DataFrame(movies_data, columns = ["Title", "Genre", "Stars", "Runtime", "MetaScore", "Votes", "Gross_value"])
df.head()

,Title,Genre,Stars,Runtime,MetaScore,Votes,Gross_value
0,Bullet Train,"Action, Comedy, Thriller",7.3,127 min,49,"424,602",$103.37M
1,Emancipation,"Action, Thriller",6.2,132 min,53,"27,028",N/A
2,Violent Night,"Action, Comedy, Thriller",6.7,112 min,55,"98,771",$50.06M
3,Top Gun: Maverick,"Action, Drama",8.2,130 min,N/A,"692,609",$718.73M
4,The Batman,"Action, Crime, Drama",7.8,176 min,N/A,"784,287",$369.35M


In [25]:
df.to_csv("movies.csv", index=False)

In [26]:
print("Shape: ",df.shape)

Shape:  (100, 7)


In [27]:
df.describe(include="all")

,Title,Genre,Stars,Runtime,MetaScore,Votes,Gross_value
count,100,100,100,100,100,100,100
unique,100,32,44,42,14,100,8
top,Bullet Train,"Action, Thriller",3.7,96 min,N/A,"424,602",N/A
freq,1,23,9,7,78,1,93
